In [3]:
import os
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from pathlib import Path

%load_ext autoreload
%autoreload 2

In [4]:
np.random.seed(42)

In [5]:
load_dotenv(override=True)

True

In [6]:
DIR_DATA_PROCESSED: str = os.getenv("DIR_DATA_PROCESSED")

In [7]:
df = pd.read_csv(f"{DIR_DATA_PROCESSED}/data.csv")

In [8]:
X = df.drop(columns="isFraud")
y = df["isFraud"]

In [9]:
print(f"skew = {y[y==1].shape[0]/y.shape[0]}")

skew = 0.002964544224336551
